In [5]:
import requests
import base64
import pandas as pd

In [3]:

# Replace with your Spotify API credentials
client_id = '3f544c3011e1457787cbbab24dd87b56'
client_secret = 'b8ec8869046040fc92876aae70ec0452'

# Define the Spotify token endpoint URL
token_url = 'https://accounts.spotify.com/api/token'

def get_token(client_id, client_secret, token_url):

    # Encode the client ID and client secret for Basic Authentication
    credentials = base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode()

    # Define the headers for the request
    headers = {
        'Authorization': f'Basic {credentials}'
    }

    # Define the request body with grant type
    data = {
        'grant_type': 'client_credentials'
    }

    # Make the POST request to obtain the token
    response = requests.post(token_url, headers=headers, data=data)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        token_data = response.json()
        
        # Extract the access token
        access_token = token_data['access_token']
        
        return access_token
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [4]:
def get_audio_analysis(track_id, access_token):
    # Define the Spotify API endpoint for audio analysis
    url = f'https://api.spotify.com/v1/audio-analysis/{track_id}'
    
    # Set the authorization header with the access token
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    # Make the API request
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

In [10]:
df = pd.read_csv('Data/reduced_isrc.csv', index_col=0)
df = df.sample(1)


In [11]:
# Get the access token
access_token = get_token(client_id, client_secret, token_url)

#for index, row in df.iterrows():
track_id = df['spotify_id'].values[0]

print(f"Name: {df['name'].values[0]}")
print(f"Artist: {df['artists'].values[0]}")

# Get the audio analysis
data = get_audio_analysis(track_id, access_token)

sections_cnt = len(data['sections'])
segments_cnt = len(data['segments'])

print(f"Sections: {sections_cnt}")
print(f"Segments: {segments_cnt}")


Name: border town - solo demo
Artist: jd souther
Sections: 8
Segments: 768


In [13]:
# Create a dataframe with the segments data
df_segments = pd.DataFrame(data['segments'])
df_segments

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,pitches,timbre
0,0.00000,0.32494,0.000,-60.000,0.29605,-55.636,0.0,"[1.0, 0.71, 0.482, 0.763, 0.784, 0.809, 0.76, ...","[0.231, 167.047, 7.175, -34.911, 54.439, -51.6..."
1,0.32494,0.30113,1.000,-54.911,0.13016,-30.173,0.0,"[0.046, 0.04, 0.067, 0.044, 0.121, 0.321, 0.13...","[19.34, -88.201, -19.179, -158.457, 65.445, 38..."
2,0.62608,0.81837,0.734,-42.416,0.07919,-32.003,0.0,"[0.946, 0.804, 0.841, 1.0, 0.657, 0.823, 0.401...","[18.195, 52.897, -100.43, 36.832, 20.984, -19...."
3,1.44444,0.10463,0.859,-54.630,0.05948,-45.473,0.0,"[1.0, 0.977, 0.566, 0.562, 0.456, 0.3, 0.225, ...","[12.868, -76.072, -108.745, -82.317, 61.284, -..."
4,1.54907,0.71043,1.000,-47.103,0.04081,-29.673,0.0,"[0.829, 0.514, 0.346, 0.465, 1.0, 0.731, 0.532...","[22.713, 3.197, -105.625, 9.945, 57.074, 58.59..."
...,...,...,...,...,...,...,...,...,...
763,237.39597,0.15574,0.697,-28.285,0.05639,-21.343,0.0,"[0.095, 0.053, 0.062, 0.355, 0.146, 0.144, 1.0...","[36.54, 93.811, -30.979, -47.91, 47.393, -11.6..."
764,237.55170,0.13283,0.845,-24.179,0.02771,-14.385,0.0,"[0.042, 0.042, 0.042, 0.224, 0.163, 0.452, 1.0...","[43.175, 55.82, -39.79, -19.758, 41.135, -32.7..."
765,237.68454,0.30848,0.617,-19.724,0.03646,-12.881,0.0,"[0.032, 0.078, 0.064, 0.164, 0.489, 0.723, 1.0...","[43.131, 89.074, -57.154, 15.099, 24.397, -16...."
766,237.99303,0.28707,0.918,-24.354,0.04705,-13.068,0.0,"[0.035, 0.092, 0.045, 0.287, 1.0, 0.399, 0.068...","[42.238, 107.465, -43.508, 18.217, 26.34, 2.71..."


In [14]:
# Create a dataframe with the sections data
df_sections = pd.DataFrame(data['sections'])
df_sections

,start,duration,confidence,loudness,tempo,tempo_confidence,key,key_confidence,mode,mode_confidence,time_signature,time_signature_confidence
0,0.00000,9.05926,1.000,-24.623,89.693,0.709,0,0.033,0,0.348,4,1.000
1,9.05926,22.90145,1.000,-16.474,94.313,0.621,4,0.403,1,0.426,5,0.941
2,31.96071,26.62430,0.009,-15.298,96.771,0.624,4,0.572,1,0.554,4,1.000
3,58.58501,29.76082,0.697,-12.850,97.011,0.594,11,0.050,1,0.344,4,1.000
4,88.34583,29.01117,0.792,-15.309,97.256,0.633,9,0.325,1,0.319,4,1.000
5,117.35700,49.23487,0.094,-14.665,97.229,0.585,9,0.241,1,0.121,4,1.000
6,166.59187,29.71823,0.742,-14.515,98.963,0.635,4,0.480,1,0.584,4,1.000
7,196.31010,53.84990,0.039,-14.934,100.082,0.608,4,0.487,1,0.519,4,1.000


In [15]:
#show pitch of second segment
df_segments['pitches'][1]

[0.046,
 0.04,
 0.067,
 0.044,
 0.121,
 0.321,
 0.136,
 0.03,
 0.076,
 0.311,
 1.0,
 0.442]

In [16]:
#show timbre of second segment
df_segments['timbre'][1]

[19.34,
 -88.201,
 -19.179,
 -158.457,
 65.445,
 38.278,
 11.862,
 -13.81,
 -33.556,
 57.139,
 -7.05,
 11.758]

In [45]:
# take all segments with confidence > 0.9
df_confident_segments = df_segments[df_segments['confidence'] > 0.8]

In [46]:
#for each pitch in the segment
pitches = df_confident_segments['pitches']

In [47]:
import numpy as np

def find_near_one_vectors(array_of_vectors, threshold=0.9):
    """
    Find vectors with values near 1 and others near 0.

    Args:
        array_of_vectors (list): List of vectors (each a list of length 12).
        threshold (float): Threshold value for considering a value near 1.

    Returns:
        tuple: A tuple containing:
            - count (int): Number of vectors matching the criteria.
            - positions (list of lists): Positions of values near 1 in each matching vector.
    """
    count = 0
    positions = []

    for vector in array_of_vectors:
        # Find positions where values are near 1 (greater than or equal to the threshold)
        near_one_positions = np.where(np.array(vector) >= threshold)[0]
        # If the number of near one positions is 1 (only one value near 1) and the rest are near 0
        # built vector without the near 1 value
        vector_without_near_one = np.delete(vector, near_one_positions)
        if len(near_one_positions) == 1 and all(np.array(vector_without_near_one) < 0.2):
            count += 1
            positions.append(list(near_one_positions))

    return count, positions
            
            

In [48]:
#turn the pitches into a array of vectors
pitches_vectors = pitches.apply(lambda x: list(x))
len(pitches_vectors)

343

In [49]:
find_near_one_vectors(pitches_vectors)

(7, [[6], [4], [4], [4], [9], [4], [6]])

In [42]:
def count_vectors_near_one(arrays, threshold=0.5):
    """
    Count the number of vectors where nearly every value is near 1.

    Args:
        arrays (list): List of vectors (lists) to be checked.
        threshold (float): Threshold value for considering values near 1.

    Returns:
        int: Number of vectors where nearly every value is near 1.
    """
    count = 0

    for vector in arrays:
        near_one_count = sum(1 for value in vector if value >= threshold)
        if near_one_count >= len(vector) * threshold:
            count += 1

    return count

In [43]:
count_vectors_near_one(pitches_vectors)

7

In [50]:
#print key of song
df['key'].values[0]

4